## 1. Importar librerías necesarias

In [7]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, text, inspect
import sys
sys.path.append('..')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## 2. Cargar configuración desde config.yml

In [8]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)

# Obtener configuraciones específicas
config_postgres = config['ETL_PRO']
config_sqlserver = config['SQL_SERVER_DW']

print("Configuración PostgreSQL:", config_postgres)
print("\nConfiguración SQL Server:", config_sqlserver)

Configuración PostgreSQL: {'drivername': 'postgresql', 'dbname': 'prueba', 'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432}

Configuración SQL Server: {'drivername': 'mssql+pyodbc', 'dbname': 'AdventureWorksDW2022', 'user': 'sa', 'password': 'r00t.R00T', 'host': 'localhost', 'port': 1433, 'driver': 'FreeTDS'}


## 3. Conexión a PostgreSQL (ETL_PRO)

In [9]:
# Construir URL de conexión para PostgreSQL
url_postgres = (
    f"{config_postgres['drivername']}://"
    f"{config_postgres['user']}:{config_postgres['password']}@"
    f"{config_postgres['host']}:{config_postgres['port']}/"
    f"{config_postgres['dbname']}"
)

# Crear engine de SQLAlchemy para PostgreSQL
engine_postgres = create_engine(url_postgres)

# Verificar conexión
try:
    with engine_postgres.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()
        print("Conectado exitosamente a PostgreSQL")
        print(f"Versión: {version[0]}")
except Exception as e:
    print(f"Error al conectar a PostgreSQL: {e}")

Conectado exitosamente a PostgreSQL
Versión: PostgreSQL 17.6 (Debian 17.6-2.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit


## 4. Conexión a SQL Server (AdventureWorksDW2022)

In [11]:
# Construir URL de conexión para SQL Server
# Nota: Asegúrate de tener instalado pyodbc y el driver ODBC para SQL Server
url_sqlserver = (
    f"{config_sqlserver['drivername']}://"
    f"{config_sqlserver['user']}:{config_sqlserver['password']}@"
    f"{config_sqlserver['host']}:{config_sqlserver['port']}/"
    f"{config_sqlserver['dbname']}"
    f"?driver={config_sqlserver['driver'].replace(' ', '+')}"
)

# Crear engine de SQLAlchemy para SQL Server
engine_sqlserver = create_engine(url_sqlserver)

# Verificar conexión
try:
    with engine_sqlserver.connect() as conn:
        result = conn.execute(text("SELECT @@VERSION;"))
        version = result.fetchone()
        print("Conectado exitosamente a SQL Server")
        print(f"Versión: {version[0][:100]}...")
except Exception as e:
    print(f"Error al conectar a SQL Server: {e}")

Conectado exitosamente a SQL Server
Versión: Microsoft SQL Server 2022 (RTM-CU21) (KB5065865) - 16.0.4215.2 (X64) 
	Aug 11 2025 13:24:21 
	Copyri...


## 5. Ejemplo: Listar tablas en ambas bases de datos

In [12]:
# Listar tablas en PostgreSQL
print("=" * 50)
print("TABLAS EN POSTGRESQL (ETL_PRO)")
print("=" * 50)
inspector_pg = inspect(engine_postgres)
tables_pg = inspector_pg.get_table_names()
for i, table in enumerate(tables_pg, 1):
    print(f"{i}. {table}")

print(f"\nTotal: {len(tables_pg)} tablas")

# Listar tablas en SQL Server
print("\n" + "=" * 50)
print("TABLAS EN SQL SERVER (AdventureWorksDW2022)")
print("=" * 50)
try:
    inspector_ss = inspect(engine_sqlserver)
    tables_ss = inspector_ss.get_table_names()
    for i, table in enumerate(tables_ss, 1):
        print(f"{i}. {table}")
    print(f"\nTotal: {len(tables_ss)} tablas")
except Exception as e:
    print(f"Error al listar tablas de SQL Server: {e}")

TABLAS EN POSTGRESQL (ETL_PRO)
1. dim_product
2. dim_date
3. dim_salesterritory
4. dim_salesreason
5. dim_currency
6. dim_promotion
7. dim_customer
8. dim_employee
9. dim_geography
10. dim_reseller
11. fact_internet_sales
12. fact_internet_sales_reason

Total: 12 tablas

TABLAS EN SQL SERVER (AdventureWorksDW2022)
1. AdventureWorksDWBuildVersion
2. DatabaseLog
3. DimAccount
4. DimCurrency
5. DimCustomer
6. DimDate
7. DimDepartmentGroup
8. DimEmployee
9. DimGeography
10. DimOrganization
11. DimProduct
12. DimProductCategory
13. DimProductSubcategory
14. DimPromotion
15. DimReseller
16. DimSalesReason
17. DimSalesTerritory
18. DimScenario
19. FactAdditionalInternationalProductDescription
20. FactCallCenter
21. FactCurrencyRate
22. FactFinance
23. FactInternetSales
24. FactInternetSalesReason
25. FactProductInventory
26. FactResellerSales
27. FactSalesQuota
28. FactSurveyResponse
29. NewFactCurrencyRate
30. ProspectiveBuyer
31. sysdiagrams

Total: 31 tablas


## 6. Ejemplo: Ejecutar consultas SQL

Ahora puedes usar `engine_postgres` y `engine_sqlserver` para ejecutar consultas en ambas bases de datos.

In [13]:
# Ejemplo con PostgreSQL
query_pg = "SELECT * FROM information_schema.tables WHERE table_schema = 'public' LIMIT 5;"
df_postgres = pd.read_sql(query_pg, engine_postgres)
print("Ejemplo de consulta a PostgreSQL:")
print(df_postgres)

# Ejemplo con SQL Server (ajusta la consulta según tus tablas)
try:
    query_ss = "SELECT TOP 5 * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    df_sqlserver = pd.read_sql(query_ss, engine_sqlserver)
    print("\n\nEjemplo de consulta a SQL Server:")
    print(df_sqlserver)
except Exception as e:
    print(f"\n\nError al consultar SQL Server: {e}")

Ejemplo de consulta a PostgreSQL:
  table_catalog table_schema          table_name  table_type  \
0        prueba       public         dim_product  BASE TABLE   
1        prueba       public            dim_date  BASE TABLE   
2        prueba       public  dim_salesterritory  BASE TABLE   
3        prueba       public     dim_salesreason  BASE TABLE   
4        prueba       public        dim_currency  BASE TABLE   

  self_referencing_column_name reference_generation user_defined_type_catalog  \
0                         None                 None                      None   
1                         None                 None                      None   
2                         None                 None                      None   
3                         None                 None                      None   
4                         None                 None                      None   

  user_defined_type_schema user_defined_type_name is_insertable_into is_typed  \
0            

In [19]:
# Verificar que los datos en dim_product (postgres) y DimProduct (sqlserver) sean iguales

# Extraer datos de ambas bases de datos
df_postgres_product = pd.read_sql("SELECT * FROM dim_product ORDER BY productkey", engine_postgres)
df_sqlserver_product = pd.read_sql("SELECT * FROM DimProduct ORDER BY ProductKey", engine_sqlserver)

print(f"Registros en PostgreSQL (dim_product): {len(df_postgres_product)}")
print(f"Registros en SQL Server (DimProduct): {len(df_sqlserver_product)}")
print("\n" + "="*80)

# Normalizar nombres de columnas para comparación (SQL Server usa PascalCase, Postgres usa snake_case)
df_sqlserver_normalized = df_sqlserver_product.copy()
df_sqlserver_normalized.columns = df_sqlserver_normalized.columns.str.lower()

# Verificar si tienen el mismo número de registros
if len(df_postgres_product) != len(df_sqlserver_normalized):
    print(f"ADVERTENCIA: Diferente numero de registros!")
    print(f"   PostgreSQL: {len(df_postgres_product)} registros")
    print(f"   SQL Server: {len(df_sqlserver_normalized)} registros")
else:
    print(f"Mismo numero de registros: {len(df_postgres_product)}")

# Verificar columnas
print(f"\n{'='*80}")
print("COLUMNAS:")
print(f"\nPostgreSQL: {list(df_postgres_product.columns)}")
print(f"\nSQL Server: {list(df_sqlserver_product.columns)}")

# Comparar columnas comunes
common_cols = set(df_postgres_product.columns) & set(df_sqlserver_normalized.columns)
print(f"\n{'='*80}")
print(f"Columnas comunes para comparar: {len(common_cols)}")
print(f"{sorted(common_cols)}")

# Lista para almacenar las filas con diferencias
rows_with_differences = []

# Realizar comparación detallada si tienen las mismas columnas
if len(common_cols) > 0:
    # Ordenar por clave primaria y resetear índice
    df_pg_sorted = df_postgres_product.sort_values('productkey').reset_index(drop=True)
    df_ss_sorted = df_sqlserver_normalized.sort_values('productkey').reset_index(drop=True)
    
    # Seleccionar solo columnas comunes
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    print(f"\n{'='*80}")
    print("COMPARACION DE DATOS:")
    
    # Comparar valores
    differences_found = False
    for col in sorted(common_cols):
        try:
            # Convertir a string para comparación más robusta (maneja NaN, tipos diferentes, etc.)
            # Usar errors='ignore' para manejar problemas de encoding
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"\nDiferencias en columna '{col}': {diff_count} registros diferentes")
                
                # Guardar información de diferencias
                diff_mask = pg_values != ss_values
                if diff_mask.any():
                    print(f"   Ejemplos (primeros 5):")
                    diff_indices = diff_mask[diff_mask].index[:5]
                    for idx in diff_indices:
                        pg_val = str(df_pg_compare.loc[idx, col])[:50]
                        ss_val = str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')[:50]
                        print(f"   - Fila {idx}: PG='{pg_val}' vs SS='{ss_val}'")
                    
                    # Agregar todas las filas con diferencias en esta columna
                    for idx in diff_mask[diff_mask].index:
                        row_diff = {
                            'fila_index': idx,
                            'productkey': df_pg_compare.loc[idx, 'productkey'],
                            'columna_diferente': col,
                            'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                            'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                        }
                        rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"\nError al comparar columna '{col}': {str(e)}")
            print(f"   Tipo en PostgreSQL: {df_pg_compare[col].dtype}")
            print(f"   Tipo en SQL Server: {df_ss_compare[col].dtype}")
    
    if not differences_found:
        print("\nLOS DATOS SON IDENTICOS! Todas las columnas comunes coinciden perfectamente.")
    else:
        print(f"\n{'='*80}")
        print("Se encontraron diferencias entre las tablas.")
        
        # Exportar diferencias a CSV
        if rows_with_differences:
            df_differences = pd.DataFrame(rows_with_differences)
            csv_filename = 'diferencias_dim_product.csv'
            df_differences.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f"\nSe exportaron {len(rows_with_differences)} diferencias al archivo: {csv_filename}")
            print(f"Total de filas unicas con diferencias: {df_differences['fila_index'].nunique()}")
            print(f"\nColumnas con diferencias:")
            print(df_differences['columna_diferente'].value_counts())
else:
    print("\nNo se encontraron columnas comunes para comparar.")

# Mostrar muestra de datos
print(f"\n{'='*80}")
print("\nMUESTRA DE DATOS (primeras 3 filas):")
print("\nPostgreSQL (dim_product):")
print(df_postgres_product.head(3))
print("\nSQL Server (DimProduct):")
# Manejar posibles problemas de encoding en la visualización
try:
    print(df_sqlserver_product.head(3))
except:
    # Si hay problemas de encoding, mostrar con columnas específicas
    print("(Nota: Algunos caracteres pueden no mostrarse correctamente debido a problemas de encoding)")
    for col in df_sqlserver_product.columns:
        print(f"\n{col}:")
        print(df_sqlserver_product[col].head(3).apply(lambda x: str(x).encode('utf-8', errors='replace').decode('utf-8')))

Registros en PostgreSQL (dim_product): 606
Registros en SQL Server (DimProduct): 606

Mismo numero de registros: 606

COLUMNAS:

PostgreSQL: ['productkey', 'productalternatekey', 'productsubcategorykey', 'weightunitmeasurecode', 'sizeunitmeasurecode', 'englishproductname', 'standardcost', 'finishedgoodsflag', 'color', 'safetystocklevel', 'reorderpoint', 'listprice', 'size', 'sizerange', 'weight', 'daystomanufacture', 'productline', 'dealerprice', 'class', 'style', 'modelname', 'largephoto', 'englishdescription', 'startdate', 'enddate', 'status', 'saved']

SQL Server: ['ProductKey', 'ProductAlternateKey', 'ProductSubcategoryKey', 'WeightUnitMeasureCode', 'SizeUnitMeasureCode', 'EnglishProductName', 'SpanishProductName', 'FrenchProductName', 'StandardCost', 'FinishedGoodsFlag', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'ListPrice', 'Size', 'SizeRange', 'Weight', 'DaysToManufacture', 'ProductLine', 'DealerPrice', 'Class', 'Style', 'ModelName', 'LargePhoto', 'EnglishDescription', 'Frenc

In [22]:
# Comparar dim_date (postgres) vs DimDate (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_date ORDER BY datekey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimDate ORDER BY DateKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('datekey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('datekey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_date',
                        'fila_index': idx,
                        'datekey': df_pg_compare.loc[idx, 'datekey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_date.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 3652 | SQL Server: 3652
Columnas comunes: 19
Diferencias en 'fiscalyear': 1829 registros
Diferencias en 'fulldatealternatekey': 3652 registros
Diferencias en 'weeknumberofyear': 731 registros

Exportadas 6212 diferencias a: diferencias_dim_date.csv
Filas unicas con diferencias: 3652
Diferencias en 'weeknumberofyear': 731 registros

Exportadas 6212 diferencias a: diferencias_dim_date.csv
Filas unicas con diferencias: 3652


In [24]:
# Comparar dim_salesterritory (postgres) vs DimSalesTerritory (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_salesterritory ORDER BY salesterritorykey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimSalesTerritory ORDER BY SalesTerritoryKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

# Verificar si tienen diferente cantidad de registros
if len(df_pg) != len(df_ss):
    print(f"\nADVERTENCIA: Diferente cantidad de registros!")
    print(f"Diferencia: {abs(len(df_pg) - len(df_ss))} registros")
    
    # Encontrar claves que están en una tabla pero no en la otra
    pg_keys = set(df_pg['salesterritorykey'])
    ss_keys = set(df_ss['salesterritorykey'])
    
    only_pg = pg_keys - ss_keys
    only_ss = ss_keys - pg_keys
    
    if only_pg:
        print(f"\nClaves solo en PostgreSQL: {sorted(only_pg)}")
    if only_ss:
        print(f"Claves solo en SQL Server: {sorted(only_ss)}")

if len(common_cols) > 0 and len(df_pg) > 0 and len(df_ss) > 0:
    df_pg_sorted = df_pg.sort_values('salesterritorykey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('salesterritorykey').reset_index(drop=True)
    
    # Comparar solo las claves que existen en ambas tablas
    common_keys = set(df_pg_sorted['salesterritorykey']) & set(df_ss_sorted['salesterritorykey'])
    
    if len(common_keys) > 0:
        df_pg_compare = df_pg_sorted[df_pg_sorted['salesterritorykey'].isin(common_keys)].reset_index(drop=True)
        df_ss_compare = df_ss_sorted[df_ss_sorted['salesterritorykey'].isin(common_keys)].reset_index(drop=True)
        
        print(f"\nComparando {len(common_keys)} claves comunes...")
        
        differences_found = False
        for col in sorted(common_cols):
            try:
                pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
                ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
                
                if not pg_values.equals(ss_values):
                    differences_found = True
                    diff_count = (pg_values != ss_values).sum()
                    print(f"Diferencias en '{col}': {diff_count} registros")
                    
                    diff_mask = pg_values != ss_values
                    for idx in diff_mask[diff_mask].index:
                        row_diff = {
                            'tabla': 'dim_salesterritory',
                            'fila_index': idx,
                            'salesterritorykey': df_pg_compare.loc[idx, 'salesterritorykey'],
                            'columna_diferente': col,
                            'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                            'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                        }
                        rows_with_differences.append(row_diff)
            except Exception as e:
                print(f"Error en columna '{col}': {str(e)}")
        
        if not differences_found:
            print("LOS DATOS SON IDENTICOS en las claves comunes!")
        else:
            if rows_with_differences:
                df_diff = pd.DataFrame(rows_with_differences)
                csv_file = 'diferencias_dim_salesterritory.csv'
                df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
                print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
                print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
    else:
        print("\nNo hay claves comunes para comparar")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 10 | SQL Server: 11
Columnas comunes: 5

ADVERTENCIA: Diferente cantidad de registros!
Diferencia: 1 registros
Claves solo en SQL Server: [11]

Comparando 10 claves comunes...
LOS DATOS SON IDENTICOS en las claves comunes!


In [25]:
# Comparar dim_salesreason (postgres) vs DimSalesReason (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_salesreason ORDER BY salesreasonkey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimSalesReason ORDER BY SalesReasonKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('salesreasonkey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('salesreasonkey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_salesreason',
                        'fila_index': idx,
                        'salesreasonkey': df_pg_compare.loc[idx, 'salesreasonkey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_salesreason.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 10 | SQL Server: 10
Columnas comunes: 3
LOS DATOS SON IDENTICOS!


In [26]:
# Comparar dim_promotion (postgres) vs DimPromotion (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_promotion ORDER BY promotionkey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimPromotion ORDER BY PromotionKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('promotionkey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('promotionkey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_promotion',
                        'fila_index': idx,
                        'promotionkey': df_pg_compare.loc[idx, 'promotionkey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_promotion.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 16 | SQL Server: 16
Columnas comunes: 4
Diferencias en 'enddate': 16 registros
Diferencias en 'startdate': 16 registros

Exportadas 32 diferencias a: diferencias_dim_promotion.csv
Filas unicas con diferencias: 16


In [28]:
# Comparar dim_employee (postgres) vs DimEmployee (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_employee ORDER BY employeekey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimEmployee ORDER BY EmployeeKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

# Verificar si tienen diferente cantidad de registros
if len(df_pg) != len(df_ss):
    print(f"\nADVERTENCIA: Diferente cantidad de registros!")
    print(f"Diferencia: {abs(len(df_pg) - len(df_ss))} registros")
    
    # Encontrar claves que están en una tabla pero no en la otra
    pg_keys = set(df_pg['employeekey'])
    ss_keys = set(df_ss['employeekey'])
    
    only_pg = pg_keys - ss_keys
    only_ss = ss_keys - pg_keys
    
    if only_pg:
        print(f"\nClaves solo en PostgreSQL: {sorted(only_pg)}")
        # Agregar registros solo en PostgreSQL al archivo de diferencias
        for key in only_pg:
            rows_with_differences.append({
                'tabla': 'dim_employee',
                'fila_index': -1,
                'employeekey': key,
                'columna_diferente': 'REGISTRO_COMPLETO',
                'valor_postgres': 'EXISTE',
                'valor_sqlserver': 'NO_EXISTE'
            })
    
    if only_ss:
        print(f"Claves solo en SQL Server: {sorted(only_ss)}")
        # Agregar registros solo en SQL Server al archivo de diferencias
        for key in only_ss:
            rows_with_differences.append({
                'tabla': 'dim_employee',
                'fila_index': -1,
                'employeekey': key,
                'columna_diferente': 'REGISTRO_COMPLETO',
                'valor_postgres': 'NO_EXISTE',
                'valor_sqlserver': 'EXISTE'
            })

if len(common_cols) > 0 and len(df_pg) > 0 and len(df_ss) > 0:
    df_pg_sorted = df_pg.sort_values('employeekey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('employeekey').reset_index(drop=True)
    
    # Comparar solo las claves que existen en ambas tablas
    common_keys = set(df_pg_sorted['employeekey']) & set(df_ss_sorted['employeekey'])
    
    if len(common_keys) > 0:
        df_pg_compare = df_pg_sorted[df_pg_sorted['employeekey'].isin(common_keys)].reset_index(drop=True)
        df_ss_compare = df_ss_sorted[df_ss_sorted['employeekey'].isin(common_keys)].reset_index(drop=True)
        
        print(f"\nComparando {len(common_keys)} claves comunes...")
        
        differences_found = False
        for col in sorted(common_cols):
            try:
                pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
                ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
                
                if not pg_values.equals(ss_values):
                    differences_found = True
                    diff_count = (pg_values != ss_values).sum()
                    print(f"Diferencias en '{col}': {diff_count} registros")
                    
                    diff_mask = pg_values != ss_values
                    for idx in diff_mask[diff_mask].index:
                        row_diff = {
                            'tabla': 'dim_employee',
                            'fila_index': idx,
                            'employeekey': df_pg_compare.loc[idx, 'employeekey'],
                            'columna_diferente': col,
                            'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                            'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                        }
                        rows_with_differences.append(row_diff)
            except Exception as e:
                print(f"Error en columna '{col}': {str(e)}")
        
        if not differences_found:
            print("LOS DATOS SON IDENTICOS en las claves comunes!")
        else:
            print(f"\nSe encontraron diferencias en las claves comunes")
    else:
        print("\nNo hay claves comunes para comparar")

# Exportar diferencias a CSV (si las hay)
if rows_with_differences:
    df_diff = pd.DataFrame(rows_with_differences)
    csv_file = 'diferencias_dim_employee.csv'
    df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
    print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
    print(f"Filas unicas con diferencias: {df_diff[df_diff['fila_index'] != -1]['fila_index'].nunique() if any(df_diff['fila_index'] != -1) else 0}")
    print(f"Registros solo en una BD: {len(df_diff[df_diff['fila_index'] == -1])}")
else:
    print("\nNo se encontraron diferencias")
    
print("\nNo hay columnas comunes para comparar" if len(common_cols) == 0 else "")

Registros en PostgreSQL: 290 | SQL Server: 296
Columnas comunes: 29

ADVERTENCIA: Diferente cantidad de registros!
Diferencia: 6 registros
Claves solo en SQL Server: [291, 292, 293, 294, 295, 296]

Comparando 290 claves comunes...
Diferencias en 'baserate': 267 registros
Diferencias en 'birthdate': 290 registros
Diferencias en 'departmentname': 152 registros
Diferencias en 'emailaddress': 288 registros
Diferencias en 'emergencycontactname': 288 registros
Diferencias en 'emergencycontactphone': 288 registros
Diferencias en 'employeenationalidalternatekey': 288 registros
Diferencias en 'enddate': 10 registros
Diferencias en 'firstname': 287 registros
Diferencias en 'gender': 118 registros
Diferencias en 'hiredate': 290 registros
Diferencias en 'lastname': 287 registros
Diferencias en 'loginid': 288 registros
Diferencias en 'maritalstatus': 147 registros
Diferencias en 'middlename': 278 registros
Diferencias en 'payfrequency': 132 registros
Diferencias en 'phone': 288 registros
Diferencia

In [29]:
# Comparar dim_reseller (postgres) vs DimReseller (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_reseller ORDER BY resellerkey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimReseller ORDER BY ResellerKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('resellerkey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('resellerkey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_reseller',
                        'fila_index': idx,
                        'resellerkey': df_pg_compare.loc[idx, 'resellerkey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_reseller.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 701 | SQL Server: 701
Columnas comunes: 16
Diferencias en 'addressline1': 26 registros
Diferencias en 'annualrevenue': 701 registros
Diferencias en 'annualsales': 701 registros
Diferencias en 'businesstype': 470 registros
Diferencias en 'geographykey': 95 registros
Diferencias en 'minpaymentamount': 191 registros
Diferencias en 'minpaymenttype': 569 registros
Diferencias en 'phone': 38 registros
Diferencias en 'resellername': 4 registros
Diferencias en 'yearopened': 701 registros

Exportadas 3496 diferencias a: diferencias_dim_reseller.csv
Filas unicas con diferencias: 701


In [35]:
# Comparar fact_internet_sales_reason (postgres) vs FactInternetSalesReason (sqlserver)

df_pg = pd.read_sql("SELECT * FROM fact_internet_sales_reason ORDER BY salesordernumber, salesorderlinenumber, salesreasonid", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM FactInternetSalesReason ORDER BY SalesOrderNumber, SalesOrderLineNumber, SalesReasonKey", engine_sqlserver)

# Rename salesreasonid to salesreasonkey in PostgreSQL dataframe for consistency
df_pg.rename(columns={'salesreasonid': 'salesreasonkey'}, inplace=True)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values(['salesordernumber', 'salesorderlinenumber', 'salesreasonkey']).reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values(['salesordernumber', 'salesorderlinenumber', 'salesreasonkey']).reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index[:100]:  # Limitar a 100 diferencias por columna
                    row_diff = {
                        'tabla': 'fact_internet_sales_reason',
                        'fila_index': idx,
                        'salesordernumber': df_pg_compare.loc[idx, 'salesordernumber'],
                        'salesorderlinenumber': df_pg_compare.loc[idx, 'salesorderlinenumber'],
                        'salesreasonkey': df_pg_compare.loc[idx, 'salesreasonkey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_fact_internet_sales_reason.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 64515 | SQL Server: 64515
Columnas comunes: 3
LOS DATOS SON IDENTICOS!
LOS DATOS SON IDENTICOS!


In [36]:
# Comparar fact_internet_sales (postgres) vs FactInternetSales (sqlserver)

df_pg = pd.read_sql("SELECT * FROM fact_internet_sales ORDER BY salesordernumber, salesorderlinenumber", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM FactInternetSales ORDER BY SalesOrderNumber, SalesOrderLineNumber", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values(['salesordernumber', 'salesorderlinenumber']).reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values(['salesordernumber', 'salesorderlinenumber']).reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index[:100]:  # Limitar a 100 diferencias por columna
                    row_diff = {
                        'tabla': 'fact_internet_sales',
                        'fila_index': idx,
                        'salesordernumber': df_pg_compare.loc[idx, 'salesordernumber'],
                        'salesorderlinenumber': df_pg_compare.loc[idx, 'salesorderlinenumber'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_fact_internet_sales.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 60398 | SQL Server: 60398
Columnas comunes: 26
Diferencias en 'duedate': 60398 registros
Diferencias en 'duedatekey': 60398 registros
Diferencias en 'extendedamount': 1075 registros
Diferencias en 'duedate': 60398 registros
Diferencias en 'duedatekey': 60398 registros
Diferencias en 'extendedamount': 1075 registros
Diferencias en 'freight': 50730 registros
Diferencias en 'freight': 50730 registros
Diferencias en 'orderdate': 60398 registros
Diferencias en 'orderdatekey': 60398 registros
Diferencias en 'productkey': 16605 registros
Diferencias en 'orderdate': 60398 registros
Diferencias en 'orderdatekey': 60398 registros
Diferencias en 'productkey': 16605 registros
Diferencias en 'productstandardcost': 2449 registros
Diferencias en 'revisionnumber': 60398 registros
Diferencias en 'salesamount': 1075 registros
Diferencias en 'productstandardcost': 2449 registros
Diferencias en 'revisionnumber': 60398 registros
Diferencias en 'salesamount': 1075 registros
Diferenc

In [37]:
# Comparar dim_geography (postgres) vs DimGeography (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_geography ORDER BY geographykey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimGeography ORDER BY GeographyKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('geographykey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('geographykey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_geography',
                        'fila_index': idx,
                        'geographykey': df_pg_compare.loc[idx, 'geographykey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_geography.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 655 | SQL Server: 655
Columnas comunes: 8
Diferencias en 'city': 12 registros
Diferencias en 'ipaddresslocator': 402 registros
Diferencias en 'postalcode': 85 registros
Diferencias en 'stateprovincecode': 565 registros

Exportadas 1064 diferencias a: diferencias_dim_geography.csv
Filas unicas con diferencias: 601


In [38]:
# Comparar dim_customer (postgres) vs DimCustomer (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_customer ORDER BY customerkey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimCustomer ORDER BY CustomerKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('customerkey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('customerkey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_customer',
                        'fila_index': idx,
                        'customerkey': df_pg_compare.loc[idx, 'customerkey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_customer.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 18484 | SQL Server: 18484
Columnas comunes: 21
Diferencias en 'addressline1': 2 registros
Diferencias en 'birthdate': 18484 registros
Diferencias en 'datefirstpurchase': 18484 registros
Diferencias en 'datefirstpurchase': 18484 registros
Diferencias en 'geographykey': 1770 registros
Diferencias en 'yearlyincome': 18484 registros
Diferencias en 'geographykey': 1770 registros
Diferencias en 'yearlyincome': 18484 registros

Exportadas 57224 diferencias a: diferencias_dim_customer.csv
Filas unicas con diferencias: 18484

Exportadas 57224 diferencias a: diferencias_dim_customer.csv
Filas unicas con diferencias: 18484


In [39]:
# Comparar dim_currency (postgres) vs DimCurrency (sqlserver)

df_pg = pd.read_sql("SELECT * FROM dim_currency ORDER BY currencykey", engine_postgres)
df_ss = pd.read_sql("SELECT * FROM DimCurrency ORDER BY CurrencyKey", engine_sqlserver)

print(f"Registros en PostgreSQL: {len(df_pg)} | SQL Server: {len(df_ss)}")

df_ss.columns = df_ss.columns.str.lower()
common_cols = set(df_pg.columns) & set(df_ss.columns)
print(f"Columnas comunes: {len(common_cols)}")

rows_with_differences = []

if len(common_cols) > 0:
    df_pg_sorted = df_pg.sort_values('currencykey').reset_index(drop=True)
    df_ss_sorted = df_ss.sort_values('currencykey').reset_index(drop=True)
    
    df_pg_compare = df_pg_sorted[sorted(common_cols)]
    df_ss_compare = df_ss_sorted[sorted(common_cols)]
    
    differences_found = False
    for col in sorted(common_cols):
        try:
            pg_values = df_pg_compare[col].apply(lambda x: str(x) if pd.notna(x) else 'NaN')
            ss_values = df_ss_compare[col].apply(lambda x: str(x).encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else 'NaN')
            
            if not pg_values.equals(ss_values):
                differences_found = True
                diff_count = (pg_values != ss_values).sum()
                print(f"Diferencias en '{col}': {diff_count} registros")
                
                diff_mask = pg_values != ss_values
                for idx in diff_mask[diff_mask].index:
                    row_diff = {
                        'tabla': 'dim_currency',
                        'fila_index': idx,
                        'currencykey': df_pg_compare.loc[idx, 'currencykey'],
                        'columna_diferente': col,
                        'valor_postgres': str(df_pg_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8'),
                        'valor_sqlserver': str(df_ss_compare.loc[idx, col]).encode('utf-8', errors='ignore').decode('utf-8')
                    }
                    rows_with_differences.append(row_diff)
        except Exception as e:
            print(f"Error en columna '{col}': {str(e)}")
    
    if not differences_found:
        print("LOS DATOS SON IDENTICOS!")
    else:
        if rows_with_differences:
            df_diff = pd.DataFrame(rows_with_differences)
            csv_file = 'diferencias_dim_currency.csv'
            df_diff.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"\nExportadas {len(rows_with_differences)} diferencias a: {csv_file}")
            print(f"Filas unicas con diferencias: {df_diff['fila_index'].nunique()}")
else:
    print("No hay columnas comunes para comparar")

Registros en PostgreSQL: 105 | SQL Server: 105
Columnas comunes: 3
Diferencias en 'currencyname': 105 registros

Exportadas 105 diferencias a: diferencias_dim_currency.csv
Filas unicas con diferencias: 105
